이 노트북은 [케라스 창시자에게 배우는 딥러닝 2판](https://tensorflow.blog/kerasdl2/)의 예제 코드를 담고 있습니다.

<table align="left">
    <tr>
        <td>
            <a href="https://colab.research.google.com/github/rickiepark/deep-learning-with-python-2nd/blob/main/chapter12_part01_text-generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
        </td>
    </tr>
</table>

# 생성 모델을 위한 딥러닝

## 텍스트 생성

### 시퀀스 생성을 위한 딥러닝 모델의 간단한 역사

### 시퀀스 데이터를 어떻게 생성할까?

### 샘플링 전략의 중요성

**다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기**

In [1]:
import numpy as np
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### 케라스를 사용한 텍스트 생성 모델 구현

#### 데이터 준비

**IMDB 영화 리뷰 데이터셋 다운로드하고 압축 풀기**

In [2]:
# !wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

**텍스트 파일(한 파일 = 한 샘플)에서 데이터셋 만들기**

In [6]:
import tensorflow as tf
from tensorflow import keras
dataset_raw = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset_raw.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 119762 files.


In [14]:
print(dataset_raw), print(dataset_raw.take(1)), print(dataset_raw.take(1).as_numpy_iterator().next())

<_PrefetchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
<_TakeDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
[b"I am a big fan of Gregg Araki's movies especially the trilogy this movie is a part of. My friends and I have even assembled our own little cult following for it. It is for sure a twisted and perverse movie but at the same time it has a message and funny all at the same time. I would only recommend this if you are into shocking, graphic commentary on our society and can stomach violence and sexual deviance. Now since the waring is out of the way I want to say i think this movie is genius and i love the WTF? feeling you get after watching this and the other two movie in this series (Nowhere and Totally F***ed Up). If you do watch this movie and don't like it the first time I suggest you watch it again, i promise after the initial shock you will see the art behind it all."
 b'It\'s hard to say which comes out on top, Jam

(None, None, None)

In [11]:
# Dataset 구조 확인
print("Dataset structure:", dataset)

# Dataset 안의 데이터 차원과 예시 출력
for batch in dataset.take(1):  # 첫 번째 배치를 확인
    print("Batch shape:", batch.shape)  # 배치의 텐서 차원 출력
    for text in batch.numpy()[:5]:  # 각 배치의 처음 5개의 텍스트를 출력
        print(text.decode('utf-8'))


Dataset structure: <_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
Batch shape: (256,)
My family watches this movie over and over. Even our 3 year old loves it. I like the "goodness" in the movie. Giving the stranger a chance...showing goodwill to one obviously in need of some unconditional acceptance. The movie gives a feeling of goodwill and victory. One other aspect of the movie that makes it so appealing is the personalities of Velvet's siblings. The bird lover. The bug lover. The boy lover! Very cute and happy movie. There is one thing, however that is irritating about it. That is how Mrs. Brown often makes Mr. Brown look foolish or unwise. She, at times,comes off as a know it all, and he as a dimwit, which he is not. Too bad to put that into such a nice story. Nevertheless, we will continue to enjoy this wonderful, old movie!
It is a pleasure to see such creativity on TV again. This show is poetic, artistic and good fun. The characters relate well

2024-06-14 05:42:23.038898: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


**`TextVectorization` 층 준비하기**

In [15]:
from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

2024-06-14 05:49:01.264156: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


**언어 모델링 데이터셋 만들기**

In [16]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

#### 트랜스포머 기반의 시퀀스-투-시퀀스 모델

In [31]:
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.cast(tf.not_equal(inputs, 0), dtype=tf.bool)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    # def get_causal_attention_mask(self, inputs):
    #     input_shape = tf.shape(inputs)
    #     batch_size, sequence_length = input_shape[0], input_shape[1]
    #     i = tf.range(sequence_length)[:, tf.newaxis]
    #     j = tf.range(sequence_length)
    #     mask = tf.cast(i >= j, dtype="int32")
    #     mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    #     mult = tf.concat(
    #         [tf.expand_dims(batch_size, -1),
    #          tf.constant([1, 1], dtype=tf.int32)], axis=0)
    #     return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # causal_mask = self.get_causal_attention_mask(inputs)
        # if mask is not None:
        #     padding_mask = tf.cast(
        #         mask[:, tf.newaxis, :], dtype="int32")
        #     padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            use_causal_mask=True)
            # attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs
            # attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**간단한 트랜스포머 기반 언어 모델**

In [32]:
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


### 가변 온도 샘플링을 사용한 텍스트 생성 콜백

**텍스트 생성 콜백**

In [ ]:
import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(predictions[0, i, :], temperature)
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

**언어 모델 훈련하기**

In [ ]:
# 코랩에서 정상 실행만 확인하기 위해 에포크 횟수를 200에서 10으로 줄입니다
model.fit(lm_dataset, epochs=10,  # 200
          callbacks=[text_gen_callback])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 5.9244== Generating with temperature 0.2
This movie movie is is a a lot movie of i the think worst that movie i i was was a a lot lot of of the the movie movie i was think a that lot i of was the not movie be i a am lot not of only the reason
== Generating with temperature 0.5
This movie movie was has very been long better time than i the dont whole understand time how i stupid watched i it am was sure [UNK] how and much watch more it than really that really it bad was but a i movie am was sure going i to have see
== Generating with temperature 0.7
This movie movie is was even an dragged interesting released documentary its because interesting i script thought is it funny has not no as longer if and it i whats love to it see was it too was bad a i hard have to been see going what through can the see
== Generating with temperature 1.0
This movie is was none filled id with always creative pick dancers i lyrics just dif

### 정리